In [3]:
using Pkg
Pkg.add("ImageMagick")
Pkg.add("FileIO")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [4]:
using Agents, Agents.Pathfinding
using Random
import ImageMagick
using FileIO: load

In [5]:
@multiagent :opt_speed struct Animal(ContinuousAgent{3,Float64})
    @subagent struct Rabbit
        energy::Float64
    end
    @subagent struct Fox
        energy::Float64
    end
    @subagent struct Hawk
        energy::Float64
    end
end

LoadError: LoadError: MethodError: no method matching var"@multiagent"(::LineNumberNode, ::Module, ::QuoteNode, ::Expr)
The function `@multiagent` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  var"@multiagent"(::LineNumberNode, ::Module, ::Any)
[0m[90m   @[39m [35mAgents[39m [90m~/.julia/packages/Agents/WuWeG/src/core/[39m[90m[4magents.jl:313[24m[39m

in expression starting at In[5]:1

In [6]:
eunorm(vec) = √sum(vec .^ 2)
const v0 = (0.0, 0.0, 0.0) # we don't use the velocity field here

(0.0, 0.0, 0.0)

In [7]:
function initialize_model(
    heightmap_url =
    "https://raw.githubusercontent.com/JuliaDynamics/" *
    "JuliaDynamics/master/videos/agents/rabbit_fox_hawk_heightmap.png",
    water_level = 8,
    grass_level = 20,
    mountain_level = 35;
    n_rabbits = 160,  ## initial number of rabbits
    n_foxes = 30,  ## initial number of foxes
    n_hawks = 30,  ## initial number of hawks
    Δe_grass = 25,  ## energy gained from eating grass
    Δe_rabbit = 30,  ## energy gained from eating one rabbit
    rabbit_repr = 0.06,  ## probability for a rabbit to (asexually) reproduce at any step
    fox_repr = 0.03,  ## probability for a fox to (asexually) reproduce at any step
    hawk_repr = 0.02, ## probability for a hawk to (asexually) reproduce at any step
    rabbit_vision = 6,  ## how far rabbits can see grass and spot predators
    fox_vision = 10,  ## how far foxes can see rabbits to hunt
    hawk_vision = 15,  ## how far hawks can see rabbits to hunt
    rabbit_speed = 1.3, ## movement speed of rabbits
    fox_speed = 1.1,  ## movement speed of foxes
    hawk_speed = 1.2, ## movement speed of hawks
    regrowth_chance = 0.03,  ## probability that a patch of grass regrows at any step
    dt = 0.1,   ## discrete timestep each iteration of the model
    seed = 42,  ## seed for random number generator
)

    # Download and load the heightmap. The grayscale value is converted to `Float64` and
    # scaled from 1 to 40
    heightmap = floor.(Int, convert.(Float64, load(download(heightmap_url))) * 39) .+ 1
    # The x and y dimensions of the pathfinder are that of the heightmap
    dims = (size(heightmap)..., 50)
    # The region of the map that is accessible to each type of animal (land-based or flying)
    # is defined using `BitArrays`
    land_walkmap = BitArray(falses(dims...))
    air_walkmap = BitArray(falses(dims...))
    for i in 1:dims[1], j in 1:dims[2]
        # land animals can only walk on top of the terrain between water_level and grass_level
        if water_level < heightmap[i, j] < grass_level
            land_walkmap[i, j, heightmap[i, j]+1] = true
        end
        # air animals can fly at any height upto mountain_level
        if heightmap[i, j] < mountain_level
            air_walkmap[i, j, (heightmap[i, j]+1):mountain_level] .= true
        end
    end

    # Generate the RNG for the model
    rng = MersenneTwister(seed)

    # Note that the dimensions of the space do not have to correspond to the dimensions
    # of the pathfinder. Discretisation is handled by the pathfinding methods
    space = ContinuousSpace((100., 100., 50.); periodic = false)

    # Generate an array of random numbers, and threshold it by the probability of grass growing
    # at that location. Although this causes grass to grow below `water_level`, it is
    # effectively ignored by `land_walkmap`
    grass = BitArray(
        rand(rng, dims[1:2]...) .< ((grass_level .- heightmap) ./ (grass_level - water_level)),
    )
    properties = (
        # The pathfinder for rabbits and foxes
        landfinder = AStar(space; walkmap = land_walkmap),
        # The pathfinder for hawks
        airfinder = AStar(space; walkmap = air_walkmap, cost_metric = MaxDistance{3}()),
        Δe_grass = Δe_grass,
        Δe_rabbit = Δe_rabbit,
        rabbit_repr = rabbit_repr,
        fox_repr = fox_repr,
        hawk_repr = hawk_repr,
        rabbit_vision = rabbit_vision,
        fox_vision = fox_vision,
        hawk_vision = hawk_vision,
        rabbit_speed = rabbit_speed,
        fox_speed = fox_speed,
        hawk_speed = hawk_speed,
        heightmap = heightmap,
        grass = grass,
        regrowth_chance = regrowth_chance,
        water_level = water_level,
        grass_level = grass_level,
        dt = dt,
    )

    model = StandardABM(Animal, space; agent_step! = animal_step!,
                        model_step! = model_step!, rng, properties)

    # spawn each animal at a random walkable position according to its pathfinder
    for _ in 1:n_rabbits
        pos = random_walkable(model, model.landfinder)
        add_agent!(pos, Rabbit, model, v0, rand(abmrng(model), Δe_grass:2Δe_grass))
    end
    for _ in 1:n_foxes
        pos = random_walkable(model, model.landfinder)
        add_agent!(pos, Fox, model, v0, rand(abmrng(model), Δe_rabbit:2Δe_rabbit))
    end
    for _ in 1:n_hawks
        pos = random_walkable(model, model.airfinder)
        add_agent!(pos, Hawk, model, v0, rand(abmrng(model), Δe_rabbit:2Δe_rabbit))
    end
    return model
end

initialize_model (generic function with 5 methods)

In [8]:
function animal_step!(animal, model)
    kind = kindof(animal)
    if kind == :Rabbit
        rabbit_step!(animal, model)
    elseif kind == :Fox
        fox_step!(animal, model)
    else
        hawk_step!(animal, model)
    end
end

animal_step! (generic function with 1 method)

In [9]:
function rabbit_step!(rabbit, model)
    # Eat grass at this position, if any
    if get_spatial_property(rabbit.pos, model.grass, model) == 1
        model.grass[get_spatial_index(rabbit.pos, model.grass, model)] = 0
        rabbit.energy += model.Δe_grass
    end

    # The energy cost at each step corresponds to the amount of time that has passed
    # since the last step
    rabbit.energy -= model.dt
    # All animals die if their energy reaches 0
    if rabbit.energy <= 0
        remove_agent!(rabbit, model, model.landfinder)
        return
    end

    # Get a list of positions of all nearby predators
    predators = [
        x.pos for x in nearby_agents(rabbit, model, model.rabbit_vision) if
            kindof(x) == :fox || kindof(x) == :hawk
            ]
    # If the rabbit sees a predator and isn't already moving somewhere
    if !isempty(predators) && is_stationary(rabbit, model.landfinder)
        # Try and get an ideal direction away from predators
        direction = (0., 0., 0.)
        for predator in predators
            # Get the direction away from the predator
            away_direction = (rabbit.pos .- predator)
            # In case there is already a predator at our location, moving anywhere is
            # moving away from it, so it doesn't contribute to `direction`
            all(away_direction .≈ 0.) && continue
            # Add this to the overall direction, scaling inversely with distance.
            # As a result, closer predators contribute more to the direction to move in
            direction = direction .+ away_direction ./ eunorm(away_direction) ^ 2
        end
        # If the only predator is right on top of the rabbit
        if all(direction .≈ 0.)
            # Move anywhere
            chosen_position = random_walkable(rabbit.pos, model, model.landfinder, model.rabbit_vision)
        else
            # Normalize the resultant direction, and get the ideal position to move it
            direction = direction ./ eunorm(direction)
            # Move to a random position in the general direction of away from predators
            position = rabbit.pos .+ direction .* (model.rabbit_vision / 2.)
            chosen_position = random_walkable(position, model, model.landfinder, model.rabbit_vision / 2.)
        end
        plan_route!(rabbit, chosen_position, model.landfinder)
    end

    # Reproduce with a random probability, scaling according to the time passed each
    # step
    rand(abmrng(model)) <= model.rabbit_repr * model.dt && reproduce!(rabbit, model)

    # If the rabbit isn't already moving somewhere, move to a random spot
    if is_stationary(rabbit, model.landfinder)
        plan_route!(
            rabbit,
            random_walkable(rabbit.pos, model, model.landfinder, model.rabbit_vision),
            model.landfinder
        )
    end

    # Move along the route planned above
    move_along_route!(rabbit, model, model.landfinder, model.rabbit_speed, model.dt)
end

rabbit_step! (generic function with 1 method)

In [10]:
function fox_step!(fox, model)
    # Look for nearby rabbits that can be eaten
    food = [x for x in nearby_agents(fox, model) if kindof(x) == :rabbit]
    if !isempty(food)
        remove_agent!(rand(abmrng(model), food), model, model.landfinder)
        fox.energy += model.Δe_rabbit
    end


    # The energy cost at each step corresponds to the amount of time that has passed
    # since the last step
    fox.energy -= model.dt
    # All animals die once their energy reaches 0
    if fox.energy <= 0
        remove_agent!(fox, model, model.landfinder)
        return
    end

    # Random chance to reproduce every step
    rand(abmrng(model)) <= model.fox_repr * model.dt && reproduce!(fox, model)

    # If the fox isn't already moving somewhere
    if is_stationary(fox, model.landfinder)
        # Look for any nearby rabbits
        prey = [x for x in nearby_agents(fox, model, model.fox_vision) if kindof(x) == :rabbit]
        if isempty(prey)
            # Move anywhere if no rabbits were found
            plan_route!(
                fox,
                random_walkable(fox.pos, model, model.landfinder, model.fox_vision),
                model.landfinder,
            )
        else
            # Move toward a random rabbit
            plan_route!(fox, rand(abmrng(model), map(x -> x.pos, prey)), model.landfinder)
        end
    end

    move_along_route!(fox, model, model.landfinder, model.fox_speed, model.dt)
end

fox_step! (generic function with 1 method)

In [11]:
function hawk_step!(hawk, model)
    # Look for rabbits nearby
    food = [x for x in nearby_agents(hawk, model) if kindof(x) == :rabbit]
    if !isempty(food)
        # Eat (remove) the rabbit
        remove_agent!(rand(abmrng(model), food), model, model.airfinder)
        hawk.energy += model.Δe_rabbit
        # Fly back up
        plan_route!(hawk, hawk.pos .+ (0., 0., 7.), model.airfinder)
    end

    # The rest of the stepping function is similar to that of foxes, except hawks use a
    # different pathfinder
    hawk.energy -= model.dt
    if hawk.energy <= 0
        remove_agent!(hawk, model, model.airfinder)
        return
    end

    rand(abmrng(model)) <= model.hawk_repr * model.dt && reproduce!(hawk, model)

    if is_stationary(hawk, model.airfinder)
        prey = [x for x in nearby_agents(hawk, model, model.hawk_vision) if kindof(x) == :rabbit]
        if isempty(prey)
            plan_route!(
                hawk,
                random_walkable(hawk.pos, model, model.airfinder, model.hawk_vision),
                model.airfinder,
            )
        else
            plan_route!(hawk, rand(abmrng(model), map(x -> x.pos, prey)), model.airfinder)
        end
    end

    move_along_route!(hawk, model, model.airfinder, model.hawk_speed, model.dt)
end

hawk_step! (generic function with 1 method)

In [12]:
function reproduce!(animal, model)
    animal.energy = Float64(ceil(Int, animal.energy / 2))
    add_agent!(animal.pos, eval(kindof(animal)), model, v0, animal.energy)
end

reproduce! (generic function with 1 method)

In [13]:
function model_step!(model)
    # To prevent copying of data, obtain a view of the part of the grass matrix that
    # doesn't have any grass, and grass can grow there
    growable = view(
        model.grass,
        model.grass .== 0 .& model.water_level .< model.heightmap .<= model.grass_level,
    )
    # Grass regrows with a random probability, scaling with the amount of time passing
    # each step of the model
    growable .= rand(abmrng(model), length(growable)) .< model.regrowth_chance * model.dt
end

model_step! (generic function with 1 method)

In [14]:
model = initialize_model()


LoadError: UndefVarError: `Animal` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [13]:
using GLMakie # CairoMakie doesn't do 3D plots well

Precompiling GLMakie
        Info Given GLMakie was explicitly requested, output will be shown live 
┌ Warning:     OpenGL/GLFW wasn't loaded correctly or couldn't be initialized.
│     This likely means, you're on a headless server without having OpenGL support setup correctly.
│     Have a look at the troubleshooting section in the readme:
│     https://github.com/MakieOrg/Makie.jl/tree/master/GLMakie#troubleshooting-opengl.
└ @ GLMakie ~/.julia/packages/GLMakie/qLxLP/src/gl_backend.jl:4
ERROR: LoadError: InitError: Exception[GLFW.GLFWError(GLFW.PLATFORM_ERROR, "X11: The DISPLAY environment variable is missing"), ErrorException("glfwInit failed")]
Stacktrace:
  [1] __init__()
    @ GLFW ~/.julia/packages/GLFW/BWxfF/src/GLFW.jl:35
  [2] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1134
  [3] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1122
  [4] _include_from_serialized(pkg::Base.PkgId, path::String, ocachep

LoadError: The following 1 direct dependency failed to precompile:

GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]

Failed to precompile GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a] to "/root/.julia/compiled/v1.10/GLMakie/jl_lzTFct".
[33m[1m┌ [22m[39m[33m[1mWarning: [22m[39m    OpenGL/GLFW wasn't loaded correctly or couldn't be initialized.
[33m[1m│ [22m[39m    This likely means, you're on a headless server without having OpenGL support setup correctly.
[33m[1m│ [22m[39m    Have a look at the troubleshooting section in the readme:
[33m[1m│ [22m[39m    https://github.com/MakieOrg/Makie.jl/tree/master/GLMakie#troubleshooting-opengl.
[33m[1m└ [22m[39m[90m@ GLMakie ~/.julia/packages/GLMakie/qLxLP/src/gl_backend.jl:4[39m
[91m[1mERROR: [22m[39mLoadError: InitError: Exception[GLFW.GLFWError(GLFW.PLATFORM_ERROR, "X11: The DISPLAY environment variable is missing"), ErrorException("glfwInit failed")]
Stacktrace:
  [1] [0m[1m__init__[22m[0m[1m([22m[0m[1m)[

In [15]:
function animalcolor(a)
    if kindof(a) === :Rabbit
        :brown
    elseif kindof(a) === :Fox
        :orange
    else
        :blue
    end
end

animalcolor (generic function with 1 method)

In [16]:
const ABMPlot = Agents.get_ABMPlot_type()
function Agents.static_preplot!(ax::Axis3, p::ABMPlot)
    surface!(
        ax,
        (100/205):(100/205):100,
        (100/205):(100/205):100,
        p.abmobs[].model[].heightmap;
        colormap = :terrain
    )
end

LoadError: UndefVarError: `Axis3` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [17]:
model

LoadError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [18]:
abmvideo(
    "rabbit_fox_hawk.mp4",
    model;
    figure = (size = (800, 700),),
    frames = 300,
    framerate = 15,
    agent_color = animalcolor,
    agent_size = 1.0,
    title = "Rabbit Fox Hawk with pathfinding"
)

LoadError: UndefVarError: `model` not defined in `Main`
Suggestion: check for spelling errors or missing imports.